In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

# nltk.download()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dkiva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data


def convert_lower_case(data):
    return np.char.lower(data)


def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)


def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data



def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)


def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data



def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)


def preprocess(data, query):
    if not query:
        data = remove_header(data)
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = stemming(data)
    return data

In [19]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0

# with open("./corpuses/sonnets.txt", 'r') as f:
    # text = f.read().strip()

for sonnet in os.listdir('./corpuses/sonnets/'):
    with open('./corpuses/sonnets/' + sonnet, 'r') as f:
        text = f.read().strip()
    preprocessed_text = preprocess(text, False)
    if doc%100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))

    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]

            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)

        pos += 1
    doc += 1

No Header
0
No Header


C:\Users\dkiva\.jpenv\lib\site-packages\IPython\core\interactiveshell.py:3361: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header
No Header


KeyboardInterrupt: 

In [13]:
# print(postings)
print(frequency)

   discas  cool  quench  disarm  virgin  legion  votari  trip  chast  nymph  \
0       1     1       1       1       1       1       1     1      1      1   

   ...  beauti  therebi  that  increas  desir  we  creatur  fairest  from  one  
0  ...       1        1     1        1      1   1        1        1     1    1  

[1 rows x 2431 columns]


In [10]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])



def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}


def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init


def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)



def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)

    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]

    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs



def print_file(file):
    out_file = open(file, 'r', encoding='utf-8')
    out_text = out_file.read()
    print(out_text)


In [16]:
get_word_postings("lively")
get_word_postings("king")
get_word_postings("time")

live
Frequency: 1
Postings List: [[0, {641, 3460, 2054, 397, 17296, 7578, 9373, 4001, 548, 3879, 6055, 7465, 6189, 7597, 10417, 8116, 439, 12090, 1726, 4159, 7614, 9156, 327, 7499, 14027, 16333, 6095, 10449, 1107, 1365, 14296, 2138, 6235, 8922, 7518, 2015, 1761, 9059, 7140, 8038, 14438, 7529, 10603, 11884, 1902, 4847, 4336, 1780, 7543, 2424, 16506, 1151}]]
king
Frequency: 1
Postings List: [[0, {3280, 12969, 9837, 7087}]]
time
Frequency: 1
Postings List: [[0, {12291, 9229, 6163, 20, 11287, 2074, 11301, 11311, 7232, 4174, 2127, 6748, 606, 6756, 614, 1643, 627, 13941, 13945, 1661, 11904, 6790, 1169, 10899, 4247, 7319, 1689, 5273, 13979, 10903, 12957, 7326, 7330, 7845, 12990, 11967, 14020, 1745, 13521, 1240, 6374, 5868, 237, 6381, 3312, 1795, 4378, 1309, 4384, 13094, 12075, 1331, 323, 14148, 8517, 5457, 5461, 2393, 6491, 1899, 5489, 14195, 4982, 13183, 7049, 5520, 6554, 7104, 12227, 2000, 8145, 3545, 8666, 479, 2026, 13808, 7153, 11257, 12287}]]
